In [259]:
import importlib
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable

import mysklearn.myruleminer
importlib.reload(mysklearn.myruleminer)
from mysklearn.myruleminer import MyAssociationRuleMiner

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import notebookutils
importlib.reload(notebookutils)
import notebookutils as nbutl

# Classification Results

In [260]:
league_standing_pytable = MyPyTable()
league_standing_pytable.load_from_file("lib/nhl_leaguestandings.csv")

league_standing_pytable.remove_rows_with_missing_values()

league_standing_pytable.drop_col("ROW")
league_standing_pytable.drop_col("RPt%")
league_standing_pytable.drop_col("OL")
league_standing_pytable.drop_col("T")
league_standing_pytable.drop_col("SEASON_TM")
league_standing_pytable.drop_col("SEASON")
league_standing_pytable.drop_col("CONFERENCE")
league_standing_pytable.drop_col("DIVISION")
league_standing_pytable.drop_col("TEAM")
league_standing_pytable.drop_col("TM")
league_standing_pytable.drop_col("PLAYOFFS")



#nbutl.discretize_ptspercent(league_standing_pytable.data, )

#'GP', 'W', 'L','PTS','PTS%', 'GF', 'SRS'
"""
key = league_standing_pytable.get_key(league_standing_pytable.column_names, ['GP', 'W', 'L','PTS','PTS%', 'GF', 'SRS'])
nbutl.discretize_gf(league_standing_pytable.data, key[0])
nbutl.discretize_srs(league_standing_pytable.data, key[1])
nbutl.discretize_gf(league_standing_pytable.data, key[2])
nbutl.discretize_gf(league_standing_pytable.data, key[3])
nbutl.discretize_ptspercent(league_standing_pytable.data, key[4])
nbutl.discretize_srs(league_standing_pytable.data, key[6])
nbutl.discretize_gf(league_standing_pytable.data, key[5])
nbutl.discretize_gf(league_standing_pytable.data, key[5]+1)
"""

y_col_name = "FINISH"

y = league_standing_pytable.get_column(y_col_name)
league_standing_pytable.drop_col(y_col_name)
X = [league_standing_pytable.data[i] for i in range(len(league_standing_pytable.data))]


X_train_folds, X_test_folds  = myevaluation.stratified_kfold_cross_validation(X, y, 10)



In [261]:
from tabulate import tabulate

# descision tree

#['GP','W','L','PTS','PTS%','GF','GA','SOS']

print("===========================================")
print("DESCISION TREE")
print("===========================================")

true_tr_strat, pred_tr_strat = notebookutils.kfoldstrat(league_standing_pytable, X, y, "tree",['GP','W','L','PTS','PTS%','GF','GA','SRS','SOS'])

tr_accuracy_strat = myevaluation.accuracy_score(true_tr_strat, pred_tr_strat)

print("Descision Tree Classifier")
print("     Accuracy = " + str(round(tr_accuracy_strat,3)) + ", Error Rate = " + str(round(1-tr_accuracy_strat,3)))
print("     Precision = " + str(round(myevaluation.binary_precision_score(true_tr_strat,pred_tr_strat),3)))
print("     Recall = " + str(round(myevaluation.binary_recall_score(true_tr_strat,pred_tr_strat),3)))
print("     F1 Score = " + str(round(myevaluation.binary_f1_score(true_tr_strat,pred_tr_strat),3)))
print("     Confusion Matrix\n" + tabulate(notebookutils.matrix(true_tr_strat, pred_tr_strat), headers=["Outcome","0","1", "2", "3", "4", "5", "Total", "Recognition(%)"]))


DESCISION TREE
Descision Tree Classifier
     Accuracy = 0.484, Error Rate = 0.516
     Precision = 0.417
     Recall = 0.326
     F1 Score = 0.366
     Confusion Matrix
  Outcome    0    1    2    3    4    5    Total    Recognition(%)
---------  ---  ---  ---  ---  ---  ---  -------  ----------------
        0  430   62   11    3    2    0      508              84.6
        1   68  201   33   15    4    3      324              62
        2   55  107   27   13    1    3      206              13.1
        3   38   53   22   28    9    6      156              17.9
        4   17   32   11   14    2    2       78               2.6
        5   14   27   11   13    2   11       78              14.1


In [262]:

print("===========================================")
print("NAIVE BAYES")
print("===========================================")


true_nb_strat, pred_nb_strat = notebookutils.kfoldstrat(league_standing_pytable, X, y, "naive",['GP','W','L','PTS','PTS%','GF','GA','SRS','SOS'])


nb_accuracy_strat = myevaluation.accuracy_score(true_nb_strat, pred_nb_strat)


print("Naive Bayes Classifier")
print("     Accuracy = " + str(round(nb_accuracy_strat,3)) + ", Error Rate = " + str(round(1-nb_accuracy_strat,3)))
print("     Precision = " + str(round(myevaluation.binary_precision_score(true_nb_strat,pred_nb_strat),3)))
print("     Recall = " + str(round(myevaluation.binary_recall_score(true_nb_strat,pred_nb_strat),3)))
print("     F1 Score = " + str(round(myevaluation.binary_f1_score(true_nb_strat,pred_nb_strat),3)))
print("     Confusion Matrix\n" + tabulate(notebookutils.matrix(true_nb_strat, pred_nb_strat), headers=["Outcome","0","1", "2", "3", "4", "5", "Total", "Recognition(%)"]))

NAIVE BAYES
Naive Bayes Classifier
     Accuracy = 0.444, Error Rate = 0.556
     Precision = 0.293
     Recall = 0.513
     F1 Score = 0.373
     Confusion Matrix
  Outcome    0    1    2    3    4    5    Total    Recognition(%)
---------  ---  ---  ---  ---  ---  ---  -------  ----------------
        0  346  143   14    4    1    0      508              68.1
        1   44  223   48    4    3    2      324              68.8
        2   25  156   16    6    2    1      206               7.8
        3    9  113   18   12    1    3      156               7.7
        4    3   58   10    4    2    1       78               2.6
        5    0   68    5    4    0    1       78               1.3


In [263]:
print("===========================================")
print("kNN CLASSIFIER")
print("===========================================")


true_knn_strat, pred_knn_strat = notebookutils.kfoldstrat(league_standing_pytable, X, y, "knn",['GP','W','L','PTS','PTS%','GF','GA','SRS','SOS'])

knn_accuracy_strat = myevaluation.accuracy_score(true_knn_strat, pred_knn_strat)


print("k Nearest Nrighbors Classifier")
print("     Accuracy = " + str(round(knn_accuracy_strat,3)) + ", Error Rate = " + str(round(1-knn_accuracy_strat,3)))
print("     Precision = " + str(round(myevaluation.binary_precision_score(true_knn_strat,pred_knn_strat),3)))
print("     Recall = " + str(round(myevaluation.binary_recall_score(true_knn_strat,pred_knn_strat),3)))
print("     F1 Score = " + str(round(myevaluation.binary_f1_score(true_knn_strat,pred_knn_strat),3)))
print("     Confusion Matrix\n" + tabulate(notebookutils.matrix(true_knn_strat, pred_knn_strat), headers=["Outcome","0","1", "2", "3", "4", "5", "Total", "Recognition(%)"]))
print("")

kNN CLASSIFIER
k Nearest Nrighbors Classifier
     Accuracy = 0.24, Error Rate = 0.76
     Precision = 0.24
     Recall = 1.0
     F1 Score = 0.387
     Confusion Matrix
  Outcome    0    1    2    3    4    5    Total    Recognition(%)
---------  ---  ---  ---  ---  ---  ---  -------  ----------------
        0    0  508    0    0    0    0      508                 0
        1    0  324    0    0    0    0      324               100
        2    0  206    0    0    0    0      206                 0
        3    0  156    0    0    0    0      156                 0
        4    0   78    0    0    0    0       78                 0
        5    0   78    0    0    0    0       78                 0



In [264]:
print("===========================================")
print("Rule Miner")
print("===========================================")

rule_miner = MyAssociationRuleMiner(minsup=0.1, minconf=.3)
rule_miner.fit(league_standing_pytable.data)
rule_miner.print_association_rules()

Rule Miner
Rule Table
#    RULE    SUPPORT    CONFIDENCE    LIFT
---  ------  ---------  ------------  ------


In [265]:
random_forest = MyRandomForestClassifier()


X_train, X_test, y_train, y_test = myevaluation.train_test_split(X, y)
random_forest.fit([X_train], [X_test], N=10, M=5, F=2)

predictions = random_forest.predict(y_train)

print(predictions)


904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904
904


TypeError: 'int' object is not iterable